### Assignment : Week 2
## Finding best policies in simple MDPs

Great work making the MDPs in Week 1!

In this assignment, we'll use the simplest RL techniques - Policy and Value iteration to find the best policies (which maximize the discounted total reward) in our MDPs from last week.

Feel free to use your own MDPs, or import them from the OpenAI Gym library.

You can start this assignment during/after reading Grokking Ch-3.

For this you have to install gymnasium, which is an API standard for reinforcement learning with a diverse collection of reference environments. This can be easily done by running:

    pip install gymnasium

In [1]:
!pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 15.2 MB/s eta 0:00:00


## Frozen Lake

Let's now try to solve the Frozen Lake environment for some cases

In [2]:
# Step 0 is to import stuff

import gymnasium as gym
import numpy as np
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

In [3]:
# Step 1 is to get the MDP
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True)
env = env.unwrapped
mdp_transitions = env.P
init_state = env.reset()
goal_state = 15

In [ ]:
# Step 2 is to write the policy

# This is according to the convention of gymnasium
LEFT, DOWN, RIGHT, UP = range(4)
pi = {
    0:RIGHT, 1:RIGHT, 2:DOWN, 3:LEFT,
    4:DOWN, 5:LEFT, 6:DOWN, 7:LEFT,
    8:RIGHT, 9:RIGHT, 10:DOWN, 11:LEFT,
    12:LEFT, 13:RIGHT, 14:RIGHT, 15:LEFT
}

# Or you can do it randomly
# pi = dict()
# for state in mdp:
#     pi[state] = np.random.choice(mdp[state].keys())

In [ ]:
# Step 3 is computing the value function for this envi and policy

# Let us start with a random value function

val = dict()
for state in mdp_transitions:
    val[state] = np.random.random()

# Since 5, 7, 11, 12 and 15 are terminal states, we know their values are 0

val[5] = 0
val[7] = 0
val[11] = 0
val[12] = 0
val[15] = 0

#Or you could do it randomly, remember to set the terminal states to 0. You can also implement this while evaluating the value function using
# val = dict()
# for state in mdp:
#     val[state] = np.random.random()
#     if mdp[state][0][0][0] == 0: # if the first action in the first outcome of the first state is 0, then it is a terminal state
#         val[state] = 0

#instead of doing thsi you can simply intialize the value function to 0 for all states
# for state in swf_mdp:
#   val[state] = 0

In [ ]:
def get_new_value_fn(val, mdp, pi, gamma=1.0):
    new_val = {}
    for state in mdp:
        action = pi[state]
        new_val[state] = sum(
            prob * (reward + gamma * val[next_state])
            for prob, next_state, reward, _ in mdp[state][action]
        )
    return new_val

In [ ]:
#Use to above function to get the new value function, also print how many iterations it took to converge
def policy_evaluation(val, mdp, pi, epsilon=1e-10, gamma=1.0):
    count = 0
    while True:
        new_val = get_new_value_fn(val, mdp, pi, gamma)
        diff = max(abs(new_val[state] - val[state]) for state in mdp)
        val = new_val
        count += 1
        if diff < epsilon:
            break
    return val, count

In [ ]:
# Perform policy improvement using the polivy and the value function and return a new policy, the action value function should be a nested dictionary
def policy_improvement(val, mdp, pi, gamma=1.0):
    new_pi = {}
    q = {}
    for state in mdp:
        q[state] = {}
        for action in range(len(mdp[state])):  # Iterate over all possible actions
            q[state][action] = sum(
                prob * (reward + gamma * val[next_state])
                for prob, next_state, reward, _ in mdp[state][action]
            )
        new_pi[state] = max(q[state], key=q[state].get)  # Choose the action with max Q-value
    return new_pi, q

In [ ]:
# Use the above functions to get the optimal policy and optimal value function and return the total number of iterations it took to converge
# Create a random policy and value function to start with or use the ones defined above
def policy_iteration(mdp, epsilon=1e-10, gamma=1.0):
    pi = {state: np.random.choice(list(mdp[state].keys())) for state in mdp}  # Random initial policy
    val = {state: 0 for state in mdp}  # Initialize value function
    count = 0

    while True:
        val, eval_count = policy_evaluation(val, mdp, pi, epsilon, gamma)
        new_pi, _ = policy_improvement(val, mdp, pi, gamma)
        count += eval_count
        if new_pi == pi:
            break
        pi = new_pi

    return pi, val, count

In [ ]:
#Now perform value iteration, note that the value function is a dictionary and not a list, also return the number of iterations it took to converge
def value_iteration(mdp, gamma=1.0, epsilon=1e-10):
    val = {state: 0 for state in mdp}
    count = 0

    while True:
        new_val = {}
        for state in mdp:
            new_val[state] = max(
                sum(prob * (reward + gamma * val[next_state])
                    for prob, next_state, reward, _ in mdp[state][action])
                for action in mdp[state]
            )
        diff = max(abs(new_val[state] - val[state]) for state in mdp)
        val = new_val
        count += 1
        if diff < epsilon:
            break

    pi = {
        state: max(
            mdp[state],
            key=lambda action: sum(
                prob * (reward + gamma * val[next_state])
                for prob, next_state, reward, _ in mdp[state][action]
            )
        )
        for state in mdp
    }

    return pi, val, count

In [ ]:
#Function to print the policy you got after running the policy iteration or value iteration on the 4x4 FrozenLake environment
def print_policy(policy, env):
    """
    Prints the policy for the 4x4 FrozenLake environment in a grid layout.
    """
    action_symbols = {0: '←', 1: '↓', 2: '→', 3: '↑'}  #action symbols
    grid_size = env.desc.shape  #get the grid dimensions (e.g., 4x4)

    policy_symbols = np.array([action_symbols[action] for cell,action in policy.items()])
    policy_grid = policy_symbols.reshape(grid_size)  #reshape into a grid

    print("Policy Grid:")
    for row in policy_grid:
        print(" ".join(row))


In [ ]:
pi1, val1, count1 = policy_iteration(mdp_transitions)
pi2, val2, count2 = value_iteration(mdp_transitions)


You can also write a function `test_policy()` to test your policy after training to find the number of times you reached the goal state

In [ ]:
def test_policy(pi, env, goal_state, num_episodes=100):
    success_count = 0
    for _ in range(num_episodes):
        state, _ = env.reset()
        done = False
        while not done:
            action = pi[state]
            state, reward, done, _, _ = env.step(action)
            if state == goal_state:
                success_count += 1
                break
    success_rate = success_count / num_episodes
    print(f"Policy Success Rate: {success_rate * 100:.2f}%")
    return success_rate